In [17]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
import numpy as np
import cv2
import os
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from numpy import load

In [18]:
label = "2_way_label"
model_name = "model_2_way.h5"
num_of_classes = 2
epochs = 20
input_feature_size = 1536
train_test_ratio = 0.1
batch_size = 1


In [19]:
img_features = load('new_img_features.npy')
bert_embeddings = load('new_bert_embeddings.npy')
print(img_features.shape)
print(bert_embeddings.shape)

(8293, 768)
(8293, 768)


In [20]:
features = pd.read_csv("data_without_features.csv")
features.shape

(8293, 5)

In [21]:
features.head()

,Unnamed: 0,img_path,2_way_label,3_way_label,6_way_label
0,0,D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\...,1.0,0.0,0.0
1,1,D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\...,0.0,2.0,1.0
2,2,D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\...,0.0,2.0,4.0
3,3,D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\...,1.0,0.0,0.0
4,4,D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\...,0.0,2.0,4.0


In [22]:
print(features.iloc[0])

Unnamed: 0                                                     0
img_path       D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\...
2_way_label                                                    1
3_way_label                                                    0
6_way_label                                                    0
Name: 0, dtype: object


In [23]:
merged_features = np.concatenate((img_features, bert_embeddings), axis=1)
merged_features.shape

(8293, 1536)

In [24]:
X = merged_features
Y = np.array(features[label])
Y = tf.keras.utils.to_categorical(Y, num_of_classes)

In [25]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=train_test_ratio, random_state=123)

In [26]:
print("x_train-",x_train.shape)
print("x_test-",x_test.shape)
print("y_train-",y_train.shape)
print("y_test-",y_test.shape)

x_train- (7463, 1536)
x_test- (830, 1536)
y_train- (7463, 2)
y_test- (830, 2)


In [27]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (7463, 1536)
7463 train samples
830 test samples


In [28]:
model = Sequential()

model.add(Dense(256, input_dim=input_feature_size,  activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_of_classes, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model Summary-",model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               393472    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 66        
Total params: 436,770
Trainable params: 436,770
Non-trainable params: 0
_________________________________________________________________
Model Summary- None


In [29]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/20
7463/7463 [==============================] - 10s 1ms/sample - loss: 0.5908 - acc: 0.6720
Epoch 2/20
7463/7463 [==============================] - 10s 1ms/sample - loss: 0.5504 - acc: 0.7103
Epoch 3/20
7463/7463 [==============================] - 10s 1ms/sample - loss: 0.5275 - acc: 0.7295
Epoch 4/20
7463/7463 [==============================] - 9s 1ms/sample - loss: 0.5196 - acc: 0.7350
Epoch 5/20
7463/7463 [==============================] - 9s 1ms/sample - loss: 0.5096 - acc: 0.7434
Epoch 6/20
7463/7463 [==============================] - 9s 1ms/sample - loss: 0.4966 - acc: 0.7490
Epoch 7/20
7463/7463 [==============================] - 9s 1ms/sample - loss: 0.4932 - acc: 0.7563
Epoch 8/20
7463/7463 [==============================] - 9s 1ms/sample - loss: 0.4915 - acc: 0.7545
Epoch 9/20
7463/7463 [==============================] - 9s 1ms/sample - loss: 0.4833 - acc: 0.7615
Epoch 10/20
7463/7463 [==============================] - 9s 1ms/sample - loss: 0.4790 - acc: 0.7638
Epoch 

In [30]:
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

830/830 [==============================] - 0s 62us/sample - loss: 0.5318 - acc: 0.7325
Test loss: 0.5317642533635518
Test accuracy: 0.7325301


In [14]:
model.save(model_name)